In [1]:
import gymnasium as gym
import os
#import gym
import numpy as np

import collections
import pickle
import tqdm

from stable_baselines3.common.buffers import ReplayBuffer


import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

from typing import Optional

import functools
import random
from copy import copy

import numpy as np
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict

from pettingzoo import AECEnv



import gymnasium


from pettingzoo.utils import agent_selector, wrappers

from gymnasium.utils import EzPickle



from statistics import NormalDist

import pygame

from typing import Any , Generic, Iterable, Iterator, TypeVar
ActionType = TypeVar("ActionType")


C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
C:\Users\31721\.conda\envs\DL_project\lib\site-packages\pkg_resources\__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-

In [2]:
from Utilities.new_models import *
import utils_gym
import env_model_class_2


from board_env import *



%load_ext autoreload
%autoreload 2

In [3]:
from IPython.display import clear_output, display

In [6]:
import numpy as np
class Hero_agent(int):
    def init_properties(self,agent_count,phases):
        #self.draw_count = 0
        self.init_win_count_iter(agent_count)
        self.init_move_count_epi(phases)
        

        
    def init_win_count_iter(self,agent_count):
        self.count_dict = {i:0 for i in range(1,agent_count+1)}
        self.count_draw_dict = {i:0 for i in range(1,agent_count+1)}
        self.draw_territory_count = 0
    def init_move_count_epi(self,phases):
        self.bad_move_count = 0
        self.bad_move_phase_count = {i:0 for i in phases}
        self.move_count =  {i:0 for i in phases}        
    
    def model_def(self, model):
        self.model =model

    def action_predict(self,data):
        return self.model.action_predict(data)
    def save_models(self):
        self.model.save_models()
    
    #def model_forward_call(self,name,kwarg):
    #    return self.model_dict[name](**kwarg)
        

a = Hero_agent(1)
a.init_properties(3,[1,2,3])

In [22]:
class DDQN_module:
    def __init__(self, qnet_config_dict, actor_config_dict,args,device,run_name,agent,writer=None):

        self.agent = agent
        self.run_name =run_name 
        self.actor_config_dict = actor_config_dict
        self.qnet_config_dict = qnet_config_dict
        self.args = args
        self.device = device
        self.writer = writer
        
        self.actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
        self.qf1 = QNetwork(**self.qnet_config_dict).to(self.device)
        self.qf1_target = QNetwork(**self.qnet_config_dict).to(self.device)
        self.target_actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
        
        
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.qf1_target.load_state_dict(self.qf1.state_dict())
        self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
        self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def action_predict(self,data):
        return self.actor(data)

    def train_write(self,data,iteration,epoch):
        #data = rb.sample(self.args.batch_size)
        qf1_a_values, qf1_loss, actor_loss = self.train(data)
        self.write(qf1_a_values, qf1_loss, actor_loss,epoch,iteration)

    def train(self,data):


        
        with torch.no_grad():
            collected_t_next = data.next_observations[:,-1]
            next_state_actions = self.target_actor(data.next_observations[:,:-1])
            qf1_next_target = self.qf1_target(data.next_observations[:,:-1], next_state_actions)

            
            next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * (self.args.gamma**(collected_t_next+1)).view(-1) * (qf1_next_target).view(-1)
    
        qf1_a_values = self.qf1(data.observations[:,:-1], data.actions).view(-1)
        qf1_loss = nn.functional.mse_loss(qf1_a_values, next_q_value)
        
        # optimize the model
        self.q_optimizer.zero_grad()
        qf1_loss.backward()
        self.q_optimizer.step()
        
        #if global_step % args.policy_frequency == 0:
        actor_loss = -self.qf1(data.observations[:,:-1], self.actor(data.observations[:,:-1])).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
    
        # update the target network
        for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        for param, target_param in zip(self.qf1.parameters(), self.qf1_target.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        return qf1_a_values, qf1_loss, actor_loss
        

    def write(self,qf1_a_values, qf1_loss, actor_loss,epoch,iteration):
        
        ind_epoch = epoch + (iteration-1)*self.args.update_epochs
        self.writer.add_scalar(f"losses/{self.agent}/qf1_values", qf1_a_values.mean().item(), ind_epoch)
        
        self.writer.add_scalar(f"losses/{self.agent}/qf1_loss", qf1_loss.item(), ind_epoch)
        self.writer.add_scalar(f"losses/{self.agent}/actor_loss", actor_loss.item(), ind_epoch)
        
    def save_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        torch.save(self.actor.state_dict(), newpath+"/actor.pt")
        torch.save(self.qf1.state_dict(), newpath+"/qf1.pt")
        torch.save(self.qf1_target.state_dict(), newpath+"/qf1_target.pt")
        torch.save(self.target_actor.state_dict(), newpath+"/target_actor.pt")  
    def load_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        self.actor.load_state_dict(torch.load(newpath+"/actor.pt"))
        self.qf1.load_state_dict(torch.load(newpath+"/qf1.pt"))
        self.qf1_target.load_state_dict(torch.load(newpath+"/qf1_target.pt"))
        self.target_actor.load_state_dict(torch.load(newpath+"/target_actor.pt")) 
        

                                          
                   
        
        

In [64]:
class evaluate_model:
    def __init__(self,Args,param_dict =dict({}),run_name="MountainCar-v0__exp1_ddqn___1__1724315565"
                 ):
        # #### Configurations

        self.args = Args()#tyro.cli(Args)
        self.param_dict = param_dict
        self.update_arg(param_dict=param_dict)

        
        #self.args.batch_size = int(self.args.num_envs * self.args.num_steps)
        self.args.minibatch_size = int(self.args.batch_size // self.args.num_minibatches)
        #self.args.num_iterations = self.args.total_timesteps // self.args.batch_size
        self.gam = self.args.gamma
        #self.args.minibatch_size = 256#128 
        self.num_steps = self.args.num_steps#120000#1000000
        self.num_iterations = self.args.num_iterations
        self.episode_time_lim = self.args.episode_time_lim
        self.num_episodes = self.args.num_episodes


        
        
        
        self.run_name = run_name

        self.writer = None

        # TRY NOT TO MODIFY: seeding
        random.seed(self.args.seed)
        np.random.seed(self.args.seed)
        torch.manual_seed(self.args.seed)
        
        torch.backends.cudnn.deterministic = self.args.torch_deterministic
        
        self.device = torch.device("cuda" if torch.cuda.is_available() and self.args.cuda else "cpu")
        
        
        self.playe_r = 1#"agent_1" #
        

        
        self.action_shape = (2,)

        self.env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
                            agent_count  = 4
                               ,use_placement_perc=True,render_=False)

        self.env = env_risk(**self.env_config)
        
        self.env.reset(seed=42)


            

        
        self.total_agents  = len(self.env.possible_agents)
        self.total_phases = len(self.env.phases)
        
        sample_obs = self.obs_converter(torch.tensor(self.env.last()[0]['observation']),
                                        num_classes = self.total_agents+1
                                       )
        
        self.ob_space_shape = sample_obs.shape #env.observation_space(playe_r)['observation'].shape
        self.action_mask_shape = self.env.observation_space(self.playe_r)['action_mask'].shape
        
        #self.device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

        
        
        self.agent_list = list(self.env.possible_agents)
        
        
        
        
       
        self.the_hero_agent = 1

        
        self.qnet_config_dict = dict(action_space = self.env.action_space(self.playe_r
                                                                         ).shape[0],
                                    ob_space=np.prod(self.ob_space_shape
                                                    )+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents -1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        self.actor_config_dict =  dict(env=self.env,
                        action_space = self.env.observation_space(self.playe_r)['action_mask'].shape[0],
                        ob_space=np.prod(self.ob_space_shape)+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents-1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        
        self.hero_agent_count = 2
        self.hero_agents_list = {i:Hero_agent(i) for i in range(1,self.hero_agent_count+1) } # this is a list , need to pass it as an argument
        
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_properties(self.total_agents,self.env.phases)        
            
            self.hero_agents_list[i].model_def(model = DDQN_module( self.qnet_config_dict, self.actor_config_dict,
                                                                   self.args,device = self.device,run_name=self.run_name,agent=i)
                                                )

            self.hero_agents_list[i].model.load_models()
            

            #self.target_actor.load_state_dict(self.actor.state_dict())
            #self.qf1_target.load_state_dict(self.qf1.state_dict())
            #self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
            #self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def update_arg(self,param_dict=dict({})):
       for i,j in param_dict.items():
           setattr(self.args,i,j)
    

    
    def obs_converter(self,  data, num_classes = 4, col =0 ):

        if col != None:
            return torch.concat((nn.functional.one_hot(data[:,col].detach().long(), 
                                                        num_classes = num_classes),
                                      data[:,~col,None]
                                ),axis=1
                               )[:,1:].to(self.device)
    
    def map_agent_phase_hot(self, data,num_classes = 3):
        return nn.functional.one_hot(torch.tensor(data),num_classes = num_classes)[1:].to(self.device)
    
    def map_agent_phase_vector(self, data,num_classes = 3):
        return nn.functional.one_hot(data[:,0].long(), 
                                                            num_classes = num_classes)[:,1:].to(self.device)

    
    def train_loop_init(self):
        self.gamma_t = {i:0 for i in self.env.possible_agents}
        
        
        self.draw_count = 0

        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_win_count_iter(self.total_agents )
        
        
        self.start_time = time.time()
        self.global_step = 0

    def reset_moves_hero_agents(self):
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_move_count_epi(self.env.phases)    
    
    def run_eval_loop(self):
        """
        ### Run training loop
        """


        env = env_risk(**(self.env_config | {"render_mode" : None, "bad_mov_penalization" : 0.01,"render_":False}))
        env.reset(42)
        
        self.train_loop_init()
        
        for iteration in range(1, self.num_iterations+1):
            self.sample(env,iteration)
            print(iteration)

    def sample(self,env,iteration):
        with torch.no_grad():
            # sample `worker_steps` from each worker
            #there are no worker steps... rather there are full episodes

            step = 0
            fault_condition = False
            clear_output(wait=True)
            phase = 0
        
            
            for episode in range(self.num_episodes):#num_episodes):
                
                total_rewards = {i:0 for i in env.possible_agents} #i can report this
                action=1
                
                if fault_condition:
                    env = env_risk(**(self.env_config | {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False#False
                                                        })
                                      )#game.env(render_mode=None)

                curren_epi = episode + (iteration-1)*self.num_episodes
                env.reset(curren_epi) #for riplication
                
                fault_condition = False
                step_count = 0
                
                self.reset_moves_hero_agents()
                is_draw = 0
                
                #draw_territory_count = 0
                #is_third = 0

                for agent in env.agent_iter():
                    e_t = env.terminations
                    if sum(e_t.values()) <(self.total_agents-1):
                        observation, reward, termination, truncation, info = env.last()
        
                        observation['observation'] =  self.obs_converter(
                                                        torch.tensor(
                                                            observation['observation']
                                                        ).to(self.device,dtype=torch.float32),
                                                        num_classes = self.total_agents+1)
                        
                        #episodes[step] = curren_epi
                        #obs[step] = observation['observation']#torch.Tensor(observation['observation']).to(self.device) #sould i not add it .... meaning this is the last observation after the player dies
                        #action_masks[step] = torch.Tensor(observation['action_mask']).to(self.device)
                        
                        #curr_agent = agent#int(agent[-1])
                        #current_agent[step] = 
                        curr_agent = agent
                        #current_phase[step] = 
                        phase = env.phase_selection
                        
                        phase_mapping = self.map_agent_phase_hot(phase,num_classes = self.total_phases).float()
                        
                        curr_agent_mapping = self.map_agent_phase_hot(int(curr_agent)-1,
                                                                      num_classes = self.total_agents 
                                                                     ).float()
                        
                        #current_troops_count[step] = torch.Tensor([env.board.agents[i].bucket for i in env.possible_agents]).to(self.device)
                    

                        model_in = torch.Tensor(torch.hstack((observation['observation'].reshape(-1),torch.tensor(observation['action_mask'].reshape(-1)).to(self.device),
                                           phase_mapping,
                                            curr_agent_mapping,
                                           torch.tensor([env.board.agents[curr_agent].bucket ]).to(self.device)))[None,:]#.repeat(3,axis = 0)
                                                ).float()
                        
                        #if e_t[curr_agent]:
                            #print('heeee')
                            
                        if termination or truncation: #this never happens ... the agent is removed from the current agent list and processed after the end of the cycle
                            
                            action = None

                            act = self.actor(torch.Tensor(model_in).to(self.device))
                            #act = self.
                            #act, logprob, _, value = agent_mod.get_action_and_value(model_in)
                            #values[step] = value.flatten() # so even if we are removing the guy ... we need to know what is the action he would 
                                                                #have taken at this point and what would have been its value
                            #actions[step] = act #even after going what would have been
                            #logprobs[step] = logprob        
                        else:
                            mask = observation["action_mask"]
                            if ( agent not in self.hero_agents_list):
        
                                
                                action = env.action_space(agent).sample()
                                part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
                                action = torch.Tensor([[[part_0],[np.around(action[1],2)]]]).to(self.device)
                                action = action[:,:,0]

                                if not( observation['action_mask'][action[:,0].long()]) : 
                                    fault_condition =True
                                    print(agent,observation['action_mask'],action[:,0].long())
                                    break


                            
                            else:
                                action = self.hero_agents_list[curr_agent].action_predict(torch.Tensor(model_in).to(self.device))
                                #action = self.actor(torch.Tensor(model_in).to(self.device))
                            #actions[step] = action
                            curr_agent_ = int(curr_agent)
        
                            if (not observation['action_mask'][action[:,0].long()]) : 
                                fault_condition =True
                                #self.faulting_player = agent

                                

                                


                                if  curr_agent_ in self.hero_agents_list:
                                    print(observation['action_mask'],action[:,0].long())
                                    print(env.board.calculated_action_mask(agent))
                                    print(model_in)
                                    break
                                    self.hero_agents_list[curr_agent_].bad_move_count+=1
                                    self.hero_agents_list[curr_agent_].bad_move_phase_count[phase]+=1  # when is the where_is_it_performing_bad_really
                                    #print('here',agent, action, observation['action_mask'])
                            
        
                            #if  curr_agent_ in self.hero_agents_list:
                            #    self.hero_agents_list[curr_agent_].move_count[int(current_phase[step][0])]+=1  
                            #if self.the_hero_agent == curr_agent:
                                #move_count[int(current_phase[step][0])]+=1        
        
        
                        #print('here',agent, action)
                        if action != None :
                            act_2 = action.detach().cpu().numpy()[0]#list([action.detach().cpu().numpy()[0][0], max(action.detach().cpu().numpy()[0][1],0.1) ])
                            act_2 = list([act_2[0], max(act_2[1],0.001) ])
                        else:
                            act_2 = action

                        
                        env.step(act_2 if action != None else None) 

                        if action != None:
                            curr_reward_list =  env.curr_rewards
                            if (step_count == (self.episode_time_lim-1)):
                                is_draw=1
                                curr_reward_list = {i:-100 for i in env.possible_agents }
        
        
                        #if action == None:
                        #    print('heeee')
                        #    rewards[step] = np.zeros(self.total_agents) # should i keep it -1? .... hm i dont think so .
                        #    dones[step] = np.zeros(self.total_agents) # frankly the guys is already done so we really dont have to do anything here.... this is the state post termination for a loser 
                        #    # but btw this is for the next agent ... action == None means in the last action the previous agent would have been removed.
                        #    #values[step] = 
                        #else:
        #
                        #    rewards_2[step] = torch.Tensor([env.curr_rewards[i] for i in env.possible_agents]).to(self.device)
                        #    curr_reward_list =  env.curr_rewards
                        #    
                        #    if (step_count == (self.episode_time_lim-1)):
                        #        is_draw=1
                        #        curr_reward_list = {i:-100 for i in env.possible_agents }
        #
                        #    for i in env.possible_agents:
                        #        if i != curr_agent:
                        #            self.gamma_t[i]+=1
                        #        else:
                        #            self.gamma_t[i] =0
        #
                        #        if (step_count == (self.episode_time_lim-1)):
                        #            cr_rew = -100
                        #            term = True
                        #        else:
                        #            cr_rew = env.curr_rewards[i]
                        #            term = env.terminations[i]
#
                        #        next_step_ = step-self.gamma_t[i]
                        #        rewards[next_step_,i-1] += (self.args.gamma**self.gamma_t[i])*cr_rew
                        #        t_next[next_step_,i-1] = self.gamma_t[i]
                        #        dones[next_step_,i-1] = torch.Tensor([term]).to(self.device) #so the panetly has to be added but attributions is really difficult
        
                        #list_curr_reward_list = np.array(list(curr_reward_list.values()))
                        
                        #if sum(curr_reward_list.values()) == -300:
                            #print('here')
                            #is_draw=1
        
                        
                        for age_i in env.possible_agents:
                            
                            total_rewards[age_i]+=curr_reward_list[age_i] #env.curr_rewards[age_i] if (step_count != episode_time_lim) else -100
                                    
                        
                        step +=1
                        self.global_step+=1
        
                    else:
                        print('done:',env.terminations,#env.terminations.values(),
                              ",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                        break    
                

In [58]:

exp_3 = dict(
exp_name = 'exp3_ddqn_lr_bs_1',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 1,
episode_time_lim = 10000,
num_episodes = 1

    
)

In [ ]:
evaluate_model

In [65]:
T = evaluate_model(Args,param_dict = exp_3,
                   run_name = 'MountainCar-v0__exp1_ddqn___1__1724315565')

In [66]:
T.run_eval_loop()

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1] tensor([29], device='cuda:0')
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 3.]],
       device='cuda:0')
1


In [73]:




hidden = T.hero_agents_list[2].model.actor.network( torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 3.]],
       device='cuda:0'))
logits = T.hero_agents_list[2].model.actor.actor_1(hidden)
probs = Categorical(logits=logits)

In [74]:
probs.probs

tensor([[0.0297, 0.0316, 0.0309, 0.0323, 0.0323, 0.0319, 0.0315, 0.0365, 0.0295,
         0.0301, 0.0312, 0.0319, 0.0310, 0.0322, 0.0318, 0.0334, 0.0311, 0.0317,
         0.0306, 0.0283, 0.0320, 0.0306, 0.0299, 0.0291, 0.0303, 0.0307, 0.0314,
         0.0315, 0.0327, 0.0315, 0.0291, 0.0316]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [39]:
class Actor_ddqn_(nn.Module):
    def __init__(self, action_space = None, ob_space=None):
        super().__init__()
        #self.fc1 = nn.Linear(ob_space, 256)
        #self.fc2 = nn.Linear(256, 256)
        #self.fc_mu = nn.Linear(256, action_space)


        self.network = nn.Sequential(
            layer_init(nn.Linear(ob_space, 256)),
            nn.GELU(),
            layer_init(nn.Linear(256, 256)),
            nn.GELU(),
            layer_init(nn.Linear(256, 64)),
            nn.GELU(),
            #layer_init(nn.Linear(64, 64)),
            #nn.GELU(),
        )
        self.actor_1 = layer_init(nn.Linear(64, action_space), std=0.01)
        self.actor_2 = nn.Sequential(layer_init(nn.Linear(64, 2), std=0.01), nn.Softmax(dim=1),)


    def forward(self, x):
        
        hidden = self.network(x)
        logits = self.actor_1(hidden)
        probs = Categorical(logits=logits)

        action_1 = probs.sample()[:,None]
        action_2 = self.actor_2(hidden)[:,[0]]

        action = torch.concat((action_1,action_2),1)

        return action#* self.action_scale + self.action_bias




In [40]:
a1 = Actor_ddqn_(action_space = 5, ob_space=5)

In [43]:
hidden = a1.network(torch.Tensor([[1,2,3,4,5]]))
logits = a1.actor_1(hidden)
probs = Categorical(logits=logits)

#probs.sample()[:,None]

In [50]:
for i in range(20):
    print(probs.sample())

tensor([3])
tensor([0])
tensor([4])
tensor([3])
tensor([4])
tensor([4])
tensor([0])
tensor([1])
tensor([4])
tensor([0])
tensor([0])
tensor([1])
tensor([0])
tensor([3])
tensor([0])
tensor([2])
tensor([4])
tensor([4])
tensor([0])
tensor([3])


In [42]:
a1(torch.Tensor([[1,2,3,4,5]]))

tensor([[0.0000, 0.4998]], grad_fn=<CatBackward0>)